In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install gdown



In [ ]:
import gdown

url = ''

output = '' # name of the file with extension

gdown.download(url, output, quiet=False)

In [ ]:
%matplotlib notebook
import torch
import torchvision 
from torchvision import transforms
import pytorch_lightning as pl
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install pydicom
!pip install torchmetrics
!pip install pytorch-lightning
  
from pathlib import Path
import pydicom
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import torch
import torchvision
from torchvision import transforms
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
labels = pd.read_csv("../input/rsnapneumoniadetectionchallenge/stage_2_train_labels.csv/stage_2_train_labels.csv")
labels.drop_duplicates("patientId",inplace=True)
labels.head()

In [ ]:
root_path = Path("../input/rsnapneumoniadetectionchallenge/stage_2_train_images")
save_path = Path("./Processed")

In [ ]:
!pip install ipympl

In [ ]:
fig , ax = plt.subplots(3,3,figsize=(9,9))
c = 0
for i in range(3):
  for j in range(3):
    patient_id = labels.patientId.iloc[c]
    dcm_path = root_path/patient_id
    dcm_path = dcm_path.with_suffix(".dcm")
    dcm = pydicom.read_file(dcm_path).pixel_array
    label = labels["Target"].iloc[c]

    ax[i][j].imshow(dcm,cmap="bone")
    ax[i][j].set_title(label)
    c+=1

In [ ]:
sums, sum_squared = 0 ,0
for c,patient_id in enumerate(tqdm(labels.patientId)):
    patient_id = labels.patientId.iloc[c]
    dcm_path = root_path/patient_id
    dcm_path = dcm_path.with_suffix(".dcm")
    dcm = pydicom.read_file(dcm_path).pixel_array/255

    dcm_array = cv2.resize(dcm,(222,224)).astype(np.float16)

    label = labels.Target.iloc[c]

    train_or_val = "train" if c <24000 else "val"

    current_save_path = save_path/train_or_val/str(label)
    current_save_path.mkdir(parents=True,exist_ok=True)
    np.save(current_save_path/patient_id,dcm_array)

    normalizer = 224*224
    if train_or_val == "train":
      sums += np.sum(dcm_array)/normalizer
      sum_squared += (dcm_array **2 ).sum()/normalizer 



In [ ]:
mean = sums / 24000
std = np.sqrt((sum_squared/24000) -mean**2)

In [ ]:
def load_file(path):
  return np.load(path).astype(np.float32)

In [ ]:
train_transforms = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean,std),
                                      transforms.RandomAffine(degrees=(-5,5),translate=(0,0.5),scale=(0.9,1.1)),
                                      transforms.RandomResizedCrop((224,224),scale=(.35,1))]
                                      
)

val_transforms = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean,std)]

                                      
)

In [ ]:
train_dataset = torchvision.datasets.DatasetFolder("./Processed/train/",loader = load_file,extensions="npy",
                                                    transform=train_transforms)
val_dataset = torchvision.datasets.DatasetFolder("./Processed/val/",loader = load_file,extensions="npy",
                                                    transform=val_transforms)


In [ ]:
fig , ax = plt.subplots(2,2,figsize=(9,9))

for i in range(2):
  for j in range(2):
    random_index = np.random.randint(0,24000)
    x_ray ,label = train_dataset[random_index]
    ax[i][j].imshow(x_ray[0],cmap="bone")
    ax[i][j].set_title(label)

In [ ]:
batch_size =64
num_workers = 64

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,num_workers=num_workers,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=batch_size,num_workers=num_workers,shuffle=False)

In [ ]:
class PneumoniaModel(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.model = torchvision.models.resnet18()
    self.model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    self.model.fc = torch.nn.Linear(in_features=512, out_features=1, bias=True)
    self.optimizer = torch.optim.Adam(self.model.parameters(),lr=1e-4)
    self.loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([3]))
    self.train_acc = torchmetrics.Accuracy()
    self.val_acc = torchmetrics.Accuracy()

  def forward(self,data):
    pred = self.model(data)
    return pred
  
  def training_step(self,batch,batch_idx):
    x_ray,label = batch
    label = label.float()
    pred = self(x_ray)[:,0]
    loss = self.loss_fn(pred,label)

    self.log("Train Loss",loss)
    self.log("Step Train acc",self.train_acc(torch.sigmoid(pred),label.int()))
    return loss

  def training_epoch_end(self,outs):
    self.log("Train acc",self.train_acc.compute())

  # -------coping--------
  def val_step(self,batch,batch_idx):
    x_ray,label = batch
    label = label.float()
    pred = self(x_ray)[:,0]
    loss = self.loss_fn(pred,label)

    self.log("Val Loss",loss)
    self.log("Step Val acc",self.val_acc(torch.sigmoid(pred),label.int()))


  def val_epoch_end(self,outs):
    self.log("Val acc",self.val_acc.compute())

  def configure_optimizers(self):
    return [self.optimizer]


In [ ]:
model = PneumoniaModel()


In [ ]:
checkpoint_callback = ModelCheckpoint(
    monitor="val_acc",
    save_top_k=10,
    mode="max"
)

In [ ]:
from gc import callbacks
gpus = 1

trainer = pl.Trainer(gpus=-1,
                     logger=TensorBoardLogger(save_dir="/content/logs"),
                     log_every_n_steps=1,
                     callbacks=checkpoint_callback,
                     max_epochs=20)

In [ ]:
trainer.fit(model,train_loader,val_loader)

# map

In [ ]:
torchvision.models.resnet18()

In [ ]:
## Now Lets use look at the model
temp_model = torchvision.models.resnet18()
temp_model

In [ ]:
temp_model.children()

In [ ]:
list(temp_model.children())

In [ ]:
list(temp_model.children())[:-2]

In [ ]:
# Converting above to pytorch model
torch.nn.Sequential(*list(temp_model.children())[:-2])

In [ ]:
class PneumoniaModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = torchvision.models.resnet18()
        self.model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.model.fc = torch.nn.Linear(in_features=512,out_features=1)
        
        
        self.feature_map = torch.nn.Sequential(*list(self.model.children())[:-2])
    # to compute the prediction let us define the forward function
    def forward(self,data):
        feature_map = self.feature_map(data)
        avg_pool_output = torch.nn.functional.adaptive_avg_pool2d(input=feature_map,output_size=(1,1))
        avg_output_flattened = torch.flatten(avg_pool_output)
        pred = self.model.fc(avg_output_flattened)
        return pred, feature_map

In [ ]:
torch.save(model.state_dict(), "./model")

In [ ]:
model = PneumoniaModel()
model.load_state_dict(torch.load("./model"),strict=False)
model.eval()

In [ ]:
model

In [ ]:
# model = PneumoniaModel.load_from_checkpoint("weights/weights_3.ckpt", strict=False)
# model = model(strict=False)
model.eval();

In [ ]:
def cam(model,img):
    with torch.no_grad():
        pred, features = model(img.unsqueeze(0))
    features = features.reshape((512,49))
    weight_params = list(model.model.fc.parameters())[0]
    weight = weight_params[0].detach()
    cam = torch.matmul(weight,features)
    cam_img = cam.reshape(7,7).cpu()
    return cam_img, torch.sigmoid(pred)

In [ ]:
def visualize(img,cam,pred):
    img = img[0]
    cam = transforms.functional.resize(cam.unsqueeze(0),(224,224))[0]
    
    fig,ax = plt.subplots(1,2)
    ax[0].imshow(img,cmap="bone")
    ax[1].imshow(img,cmap="bone")
    ax[1].imshow(img,cmap="jet",alpha=0.5)
    plt.title(pred>0.5)
    

In [ ]:
img = val_dataset[-6][0]
img.shape

In [ ]:
img = val_dataset[-6][0]
activation_map ,pred = cam(model,img)


In [ ]:
%matplotlib inline

In [ ]:
visualize(img,activation_map,pred)